In [2]:
def getSecret(secretName):
    linked_service = "ifmpmvault"
    akv_name = "ifm-vault"

    # Fetch the key from Azure Key Vault
    secretValue = mssparkutils.credentials.getSecret(
        linkedService=linked_service,
        akvName=akv_name, 
        secret=secretName)
    return secretValue

StatementMeta(sparkpoolforml, 67, 2, Finished, Available)

In [3]:
try:
    spark.sql("CREATE DATABASE dataverse_d365env_generated_data")
except:
    print("Database already exists")

spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY") #to read the dataverse datetime columns in spark3.x 

StatementMeta(sparkpoolforml, 67, 3, Finished, Available)

Database already exists


DataFrame[key: string, value: string]

In [4]:
file_system_name = getSecret('file-system-name')
data_lake_account_name = getSecret('data-lake-account-name')
basepath = f"abfss://{file_system_name}@{data_lake_account_name}.dfs.core.windows.net"

StatementMeta(sparkpoolforml, 67, 4, Finished, Available)

In [5]:
tableslist = ['connection',
    'incident',
    'msdyn_customerasset',
    'msdyn_iotalert',
    'msdyn_iotdevice',
    'msdyn_workorder']

for tablename in tableslist:
    file_path =  basepath + '/dataverse_d365_generated/' + tablename + '.csv'
    df = spark.read.load(file_path, format='csv', header=True,inferSchema=True)
    df.write.mode("overwrite").saveAsTable("dataverse_d365env_generated_data." + tablename)

StatementMeta(sparkpoolforml, 67, 5, Finished, Available)